**프로젝트 2 : 날씨 좋은 월요일 오후 세 시, 자전거 타는 사람은 몇 명?**

(1) 데이터 가져오기

In [45]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [46]:
train=pd.read_csv('../input/bike-sharing-demand/train.csv')

In [47]:
train.shape

In [48]:
train.head()

(2) datetime 컬럼을 datetime 자료형으로 변환하고 연, 월, 일, 시, 분, 초까지 6가지 컬럼 생성하기

In [49]:
train['datetime'] = pd.to_datetime(train['datetime']) # string에서 datetime 자료형으로 전환 : pd.to_datetime()

In [50]:
train['year'] = train['datetime'].dt.year # 건희님 코딩 참조
train['month'] = train['datetime'].dt.month
train['day'] = train['datetime'].dt.day
train['hour'] = train['datetime'].dt.hour
train['minute'] = train['datetime'].dt.minute
train['second'] = train['datetime'].dt.second

In [51]:
train.shape

In [52]:
train.head()

(3) year, month, day, hour, minute, second 데이터 개수 시각화하기

In [53]:
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

In [54]:
mpl.rc('font', size=15)      # 폰트 크기를 5로 설정 
sns.displot(train['count']); # 분포도 출력

In [55]:
sns.displot(np.log(train['count']))

In [56]:
plt.figure(figsize=(20,10))
plt.subplot(2,3,1)
sns.countplot(x = 'year', data=train)

plt.subplot(2,3,2)
sns.countplot(x = 'month', data=train)

plt.subplot(2,3,3)
sns.countplot(x = 'day', data=train)

plt.subplot(2,3,4)
sns.countplot(x = 'hour', data=train)

plt.subplot(2,3,5)
sns.countplot(x = 'minute', data=train)

plt.subplot(2,3,6)
sns.countplot(x = 'second',data=train)

plt.show()

In [57]:
# 스텝 1 : m행 n열 Figure 준비
mpl.rc('font', size=14)       # 폰트 크기 설정
mpl.rc('axes', titlesize=15)  # 각 축의 제목 크기 설정
figure, axes = plt.subplots(nrows=3, ncols=2) # 3행 2열 Figure 생성 
plt.tight_layout()            # 그래프 사이에 여백 확보 
figure.set_size_inches(10, 9) # 전체 Figure 크기를 10x9인치로 설정 

# 스텝 2 : 각 축에 서브플롯 할당
# 각 축에 연도, 월, 일, 시간, 분, 초별 평균 대여 수량 막대 그래프 할당
sns.barplot(x='year', y='count', data=train, ax=axes[0, 0])
sns.barplot(x='month', y='count', data=train, ax=axes[0, 1])
sns.barplot(x='day', y='count', data=train, ax=axes[1, 0])
sns.barplot(x='hour', y='count', data=train, ax=axes[1, 1])
sns.barplot(x='minute', y='count', data=train, ax=axes[2, 0])
sns.barplot(x='second', y='count', data=train, ax=axes[2, 1])

# 스텝 3 : 세부 설정
# 3-1 : 서브플롯에 제목 달기
axes[0, 0].set(title='Rental amounts by year')
axes[0, 1].set(title='Rental amounts by month')
axes[1, 0].set(title='Rental amounts by day')
axes[1, 1].set(title='Rental amounts by hour')
axes[2, 0].set(title='Rental amounts by minute')
axes[2, 1].set(title='Rental amounts by second')

# 3-2 : 1행에 위치한 서브플롯들의 x축 라벨 90도 회전
axes[1, 0].tick_params(axis='x', labelrotation=90)
axes[1, 1].tick_params(axis='x', labelrotation=90)

(4) X, y 컬럼 선택 및 train/test 데이터 분리

In [58]:
X = train[['season', 'workingday', 'weather', 'temp', 'atemp', 'humidity', 'windspeed', 'year', 'month', 'day', 'hour',]].values
y = train[['count']].values

In [59]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)

(5) LinearRegression 모델 학습

In [60]:
model = LinearRegression()

In [61]:
model.fit(X_train, y_train)

(6) 학습된 모델로 X_test에 대한 예측값 출력 및 손실함수값 계산

In [62]:
predictions = model.predict(X_test)
predictions

In [63]:
rmse = mean_squared_error(y_test, predictions, squared = False)
rmse

In [64]:
col = ['season', 'workingday', 'weather', 'temp', 'atemp', 'humidity', 'windspeed', 'year', 'month', 'day', 'hour']
X_test_df = pd.DataFrame(X_test, columns=col)
X_test_df['predictions'] = predictions
X_test_df['y_test'] = y_test

In [65]:
X_test_df

(7) x축은 temp 또는 humidity로, y축은 count로 예측 결과 시각화하기

In [66]:
plt.figure(figsize=(20,10))
plt.subplot(2,2,1)
sns.lineplot(x=X_test_df['temp'], y=X_test_df['predictions'])

plt.subplot(2,2,2)
sns.lineplot(x=X_test_df['temp'], y=X_test_df['y_test'])

plt.subplot(2,2,3)
sns.lineplot(x=X_test_df['humidity'], y=X_test_df['predictions'])

plt.subplot(2,2,4)
sns.lineplot(x=X_test_df['humidity'], y=X_test_df['y_test'])

plt.show()

**회고**

처음에 학습을 진행했을 때, loss 값이 거의 0에 가까운 값이 나왔습니다. 모델이 너무 잘 맞추는 것이 'casual', 'registered' 속성 때문이라고 판단되어, 두 개의 속성을 제거하였습니다. 추가적으로 'holiday', 'minute', 'second' 속성도 제거하여, 학습을 진행하였습니다. 결과적으로 loss 값이 140.47 정도가 나왔습니다.